In [1]:
gethostname()

"cbpro.fritz.box"

# Exercise: (Dense) Matrix-Matrix Multiplication

In this exercise you will consider dense matrix-matrix multiplication

$C_{rc} = \sum_{k=1}^{n} A_{rk} B_{kc}$

for square matrices $A$, $B$, and $C$ of size $n \times n$. Despite being seemingly simple, you will see that different implementations have vastly different performance.

## Tasks

In [2]:
# loading packages
using BenchmarkTools
using CpuId
using Test
using LinearAlgebra
BLAS.set_num_threads(1)

# problem size
const N = 512 # tune this if necessary

# input (not to be modified)
const C = zeros(N, N)
const A = rand(N, N)
const B = rand(N, N);

1) Benchmark the naive implementation `matmul_naive!` below in GFLOPS (giga floating-point operations per second).

In [3]:
function matmul_naive!(C, A, B)
    @assert size(C) == size(A) == size(B)
    @assert size(C, 1) == size(C, 2)
    n = size(C, 1)
    fill!(C, zero(eltype(C)))

    # - c: for columns of B and C
    # - r: for rows    of A and C
    # - k: for columns of A and rows of B
    for c in 1:n
        for r in 1:n
            c_reg = 0.0
            for k in 1:n
                @inbounds c_reg += A[r, k] * B[k, c]
            end
            @inbounds C[r, c] = c_reg
        end
    end
    return C
end

matmul_naive! (generic function with 1 method)

Correctness check:

In [4]:
@test matmul_naive!(C, A, B) ≈ mul!(similar(C), A, B) # can take a while

Test Passed

Benchmark:

In [5]:
t_naive = @belapsed matmul_naive!($C, $A, $B) samples = 1 evals = 2
println("matmul_naive!: ", t_naive, " sec, performance = ", round(2.0e-9 * N^3 / t_naive, digits=2), " GFLOP/s")

matmul_naive!: 0.320066767 sec, performance = 0.84 GFLOP/s


2) Why is `matmul_naive!` so (super) slow?
    * Hints: Look at the memory access pattern of innermost loop.

**Answer:** Strided memory access for `A` in `matmul_naive!` reduces the performance. The implementation doesn't take column-major order into consideration.

3) Implement an improved version with contiguous memory access (better spatial locality) in `matmul_contiguous!` and benchmark it. How much faster is it?

In [6]:
function matmul_contiguous!(C, A, B)
    @assert size(C) == size(A) == size(B)
    @assert size(C, 1) == size(C, 2)
    n = size(C, 1)
    fill!(C, zero(eltype(C)))

    # - c: for columns of B and C
    # - r: for rows    of A and C
    # - k: for columns of A and rows of B
    
    #
    # TODO: Implement improved version with more efficient memory access / better localitly.
    #       Hint: Which loop should be the innermost?
    #
    for c in 1:n
        for k in 1:n
            @inbounds b = B[k, c]
            for r in 1:n
                @inbounds C[r, c] += A[r, k] * b
            end
        end
    end
    return C
end

matmul_contiguous! (generic function with 1 method)

Correctness check:

In [7]:
@test matmul_contiguous!(C, A, B) ≈ mul!(similar(C), A, B) # this should give true, otherwise your implementation is incorrect.

Test Passed

Benchmark:

In [8]:
#
# TODO: Benchmark like matmul_naive! above
#
t_contiguous = @belapsed matmul_contiguous!($C, $A, $B) samples = 1 evals = 2
println("matmul_contiguous!: ", t_contiguous, " sec, performance = ", round(2.0e-9 * N^3 / t_contiguous, digits=2), " GFLOP/s")

matmul_contiguous!: 0.037176524 sec, performance = 7.22 GFLOP/s


### Cache Blocking

The key idea of cache blocking is to perform the overall matrix-matrix multiplication in terms of smaller matrix-matrix multiplications of sub-blocks.

<img src="../imgs/dMMM_cache_blocking.png" width=800>
<br>

In [9]:
function matmul_cache_blocking!(C, A, B; col_blksize=16, row_blksize=128, k_blksize=16)
    @assert size(C) == size(A) == size(B)
    @assert size(C, 1) == size(C, 2)
    n = size(C, 1)
    fill!(C, zero(eltype(C)))

    # - c: for columns of B and C
    # - r: for rows    of A and C
    # - k: for columns of A and rows of B
    for ic in 1:col_blksize:n
        for ir in 1:row_blksize:n
            for ik in 1:k_blksize:n
                #
                # begin: cache blocking
                #
                for jc in ic:min(ic + col_blksize - 1, n)
                    for jk in ik:min(ik + k_blksize - 1, n)
                        @inbounds b = B[jk, jc]
                        for jr in ir:min(ir + row_blksize - 1, n)
                            @inbounds C[jr, jc] += A[jr, jk] * b
                        end
                    end
                end
                #
                # end: cache blocking
                #
            end
        end
    end
    return C
end

matmul_cache_blocking! (generic function with 1 method)

In [10]:
@test matmul_cache_blocking!(C, A, B) ≈ mul!(similar(C), A, B)

Test Passed

4) Inspect the given variant `matmul_cache_blocking!` which implements cache blocking.
    * In which limit, that is, for what block size values, does the code semantically reduce to your code in 3)?
    * Staying away from these limits, how can this implementation be faster?
    
**Answer:**
* For `c_blksize = r_blksize = k_blksize = 1` and `c_blksize = r_blksize = k_blksize = n`.
* By operating on blocks that, ideally, fit into L1 or L2 cache, we maximize fast data reuse (temporal locality).

5) Benchmark the cache blocking variant. Specifically, we consider `c_blksize = 16`, `r_blksize = 128` and `k_blksize = 16` for the block sizes (default values).
    * Can you give an argument why it makes sense to choose `r_blksize` larger than the others?
    * How big (in bytes) are the blocks for `A` and `C` for these values? How does this compare to the L1 cache size of 32 KiB?
    
**Answer:**
* Julia arrays are column-major order which aligns with the row index `r`.
* Both blocks (`r_blksize * k_blksize` for A and `r_blksize * c_blksize` for C) are 16 * 128 * 8 bytes = 16 KiB big. Hence, they fit into L1 cache together.    

In [11]:
#
# TODO: Benchmark matmul_cache_blocking! like above
#
t_cache_blocking = @belapsed matmul_cache_blocking!($C, $A, $B) samples = 1 evals = 2
println("matmul_cache_blocking!: ", t_cache_blocking, " sec, performance = ", round(2.0e-9 * N^3 / t_cache_blocking, digits=2), " GFLOP/s")

matmul_cache_blocking!: 0.030870658 sec, performance = 8.7 GFLOP/s


6) Compare the performance of the cache-blocking variant to the built-in BLAS matrix multiply, i.e. `mul!(C, A, B)`.

In [12]:
#
# TODO: Benchmark mul! like above
#
t_BLAS = @belapsed mul!($C, $A, $B) samples = 1 evals = 2
println("mul! (BLAS): ", t_BLAS, " sec, performance = ", round(2.0e-9 * N^3 / t_BLAS, digits=2), " GFLOP/s")

mul! (BLAS): 0.0076120765 sec, performance = 35.26 GFLOP/s


7) **Bonus task:** vary the block sizes (in powers of 2) and see how the performance changes. (**This might take > 15 minutes.**)

In [13]:
println("Varying block sizes:")
L1 = cachesize()[1]
for cbs in (4, 8, 16, 32, 64), kbs in (4, 8, 16, 32, 64), rbs in (4, 128, 256, 512)
    if rbs * kbs + rbs * cbs > L1 / 8
        # A block and C block don't fit into L1 cache together
        continue
    end
    t_cache_block = @belapsed matmul_cache_blocking!($C, $A, $B; col_blksize=$cbs, row_blksize=$rbs, k_blksize=$kbs) samples = 1 evals = 2
    println("matmul_cache_block ($cbs, $rbs, $kbs): ", t_cache_block, " sec, performance = ", round(2.0e-9 * N^3 / t_cache_block, digits=2), " GFLOPS\n")
end

Varying block sizes:
matmul_cache_block (4, 4, 4): 0.2734503135 sec, performance = 0.98 GFLOPS

matmul_cache_block (4, 128, 4): 0.033008018 sec, performance = 8.13 GFLOPS

matmul_cache_block (4, 256, 4): 0.0280533955 sec, performance = 9.57 GFLOPS

matmul_cache_block (4, 512, 4): 0.025277515 sec, performance = 10.62 GFLOPS

matmul_cache_block (4, 4, 8): 0.2563005595 sec, performance = 1.05 GFLOPS

matmul_cache_block (4, 128, 8): 0.0322521285 sec, performance = 8.32 GFLOPS

matmul_cache_block (4, 256, 8): 0.0287662185 sec, performance = 9.33 GFLOPS

matmul_cache_block (4, 4, 16): 0.2620004395 sec, performance = 1.02 GFLOPS

matmul_cache_block (4, 128, 16): 0.033037087 sec, performance = 8.13 GFLOPS

matmul_cache_block (4, 4, 32): 0.26134012 sec, performance = 1.03 GFLOPS

matmul_cache_block (4, 4, 64): 0.285428404 sec, performance = 0.94 GFLOPS

matmul_cache_block (8, 4, 4): 0.2636320395 sec, performance = 1.02 GFLOPS

matmul_cache_block (8, 128, 4): 0.028655939 sec, performance = 9.37 